## Part 3: DBSCAN for Word Embedding Clustering

We will reference the cluster algorithm tutorial notebook Eni posted earlier and use the same example in this notebook.

In this notebook, we also attempt to replicate the results achieved by Mohammed et al. in [Glove Word Embedding and DBSCAN Algorithms for Semantic Document Clustering](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9436540&tag=1) and Eklavya Dahiya in [Exploring Unsupervised Clustering Techniques On News Articles](https://arno.uvt.nl/show.cgi?fid=160830).


First let's import/download packages we need for this example. 

In [1]:
import numpy as np
import gensim.downloader as api
from sklearn.cluster import DBSCAN
import sklearn.datasets
from nltk.tokenize import word_tokenize
import nltk

Now, let's read in the dataset. To parallel the notebooks we've worked with as tutorials in class, we'll use the 20newsgroups dataset.

In [2]:
groups = ['alt.atheism', 'comp.graphics', 'rec.autos', 
          'rec.sport.baseball', 'sci.med', 'talk.politics.guns']


train_data = sklearn.datasets.fetch_20newsgroups(subset='train', categories=groups)
test_data = sklearn.datasets.fetch_20newsgroups(subset='test', categories=groups)
print(len(train_data.filenames), len(test_data.filenames))
data = train_data.data
print(data[:10])

3395 2261
["From: david@stat.com (David Dodell)\nSubject: HICN610 Medical Newsletter, Part 2/4\nReply-To: david@stat.com (David Dodell)\nDistribution: world\nOrganization: Stat Gateway Service, WB7TPY\nLines: 708\n\n\n------------- cut here -----------------\n\n\n\n\n\n\n\n\n\nHICNet Medical Newsletter                                              Page 13\nVolume  6, Number 10                                           April 20, 1993\n\n                       Gonorrhea -- Colorado, 1985-1992\n                       ================================\n                   SOURCE: MMWR 42(14)   DATE: Apr 16, 1993\n\n     The number of reported cases of gonorrhea in Colorado increased 19.9% \nfrom 1991 to 1992 after declining steadily during the 1980s. In comparison, in \nthe United States, reported cases of gonorrhea in 1992 continued an overall \ndecreasing trend (1). This report summarizes an analysis of the increase in \ngonorrhea in Colorado in 1992 and characterizes trends in the occurren

Now we'll load our GloVe model in. [SpaCy](https://spacy.io/) is a library that embeds using GloVe by default.

In [4]:
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_lg") # Generates embeddings using GloVe

# EXAMPLE:
# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(text)
print(doc)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)


When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously. “I can tell you very senior CEOs of major American car companies would shake my hand and turn away because I wasn’t worth talking to,” said Thrun, in an interview with Recode earlier this week.
Noun phrases: ['Sebastian Thrun', 'self-driving cars', 'Google', 'few people', 'the company', 'him', 'I', 'you', 'very senior CEOs', 'major American car companies', 'my hand', 'I', 'Thrun', 'an interview', 'Recode']
Verbs: ['start', 'work', 'drive', 'take', 'tell', 'shake', 'turn', 'talk', 'say']
Sebastian Thrun PERSON
Google ORG
2007 DATE
American NORP
Thrun GPE
earlier this week DATE


In [5]:
dir(doc)

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_bulk_merge',
 '_context',
 '_get_array_attrs',
 '_realloc',
 '_vector',
 '_vector_norm',
 'cats',
 'char_span',
 'copy',
 'count_by',
 'doc',
 'ents',
 'extend_tensor',
 'from_array',
 'from_bytes',
 'from_dict',
 'from_disk',
 'from_docs',
 'from_json',
 'get_extension',
 'get_lca_matrix',
 'has_annotation',
 'has_extension',
 'has_unknown_spaces',
 'has_vector',
 'is_nered',
 'is_parsed',
 'is_sentenced',
 'is_tagged',
 'lang',
 'lang_',
 'mem',
 'noun_chunks',
 'noun_chunks_iterator',
 'remove_extension',
 'retokenize',
 'sentiment'

We define a function document_vector which takes in a document and returns the embedding of the document in vector format.

In [6]:
def document_vector(doc):
    words = nlp(doc)
    return words.vector

Then we vectorize each document.

In [7]:
vectorized = np.array([document_vector(doc) for doc in data])

In [8]:
vectorized.shape

(3395, 300)

We perform Principal Component Analysis (PCA), a technique to reduce the dimensions of the dataset without losing much information on the overall data quality. PCA is necessary for DBSCAN because scikit-learn's DBSCAN algorithm does not perform well on high dimensional data.

In [9]:
##try PCA to reduce the dimensions of the vectors as DBSCAN from sklearn is more applicable 
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
vectorized_reduced = pca.fit_transform(vectorized)

First let's arbitrarily determine the parameters for DBSCAN just to see how the code works.

In [10]:
dbscan = DBSCAN(eps=0.1, min_samples=3, metric='cosine') #use cosine similarity as metric
clusters = dbscan.fit_predict(vectorized_reduced)

In [11]:
clusters

array([-1, -1, -1, ..., -1,  0, -1])

In [12]:
import numpy as np
from collections import Counter

cluster_counts = Counter(clusters)
num_clusters = len(cluster_counts) - (1 if -1 in cluster_counts else 0)
num_noise = cluster_counts[-1] if -1 in cluster_counts else 0

print(f"Number of clusters: {num_clusters}")
print(f"Number of noise points: {num_noise}")
print(f"Cluster distribution: {dict(cluster_counts)}")

Number of clusters: 13
Number of noise points: 2224
Cluster distribution: {-1: 2224, 0: 972, 1: 3, 2: 77, 3: 5, 4: 3, 5: 85, 6: 4, 7: 4, 11: 5, 8: 3, 9: 3, 10: 4, 12: 3}


However, we can also use Silhouette Score to help us to decide on the parameters. (Credit to Josie and Sophie!)

In the paper we are attempting to replicate, they did 60 rounds with different params to determine what would be the best epsilon and min points. The code below is just a small test/demo we did that does not capture how DBSCAN iteration work in academia.

In [13]:
from sklearn.metrics import silhouette_score

for eps in np.arange(0.01, 0.5, 0.01):
    model = DBSCAN(eps=eps, min_samples=3, metric='cosine')
    labels = model.fit_predict(vectorized_reduced)
    if len(set(labels)) > 1: 
        score = silhouette_score(vectorized, labels)
        print(f"eps: {eps:.2f}, clusters: {len(set(labels))}, silhouette score: {score:.3f}")

eps: 0.01, clusters: 4, silhouette score: 0.497
eps: 0.02, clusters: 6, silhouette score: 0.506
eps: 0.03, clusters: 13, silhouette score: -0.104
eps: 0.04, clusters: 30, silhouette score: -0.227
eps: 0.05, clusters: 35, silhouette score: -0.234
eps: 0.06, clusters: 25, silhouette score: -0.153
eps: 0.07, clusters: 23, silhouette score: -0.133
eps: 0.08, clusters: 21, silhouette score: -0.088
eps: 0.09, clusters: 21, silhouette score: -0.209
eps: 0.10, clusters: 14, silhouette score: -0.181
eps: 0.11, clusters: 17, silhouette score: -0.221
eps: 0.12, clusters: 15, silhouette score: -0.116
eps: 0.13, clusters: 15, silhouette score: -0.237
eps: 0.14, clusters: 14, silhouette score: -0.248
eps: 0.15, clusters: 14, silhouette score: -0.253
eps: 0.16, clusters: 12, silhouette score: -0.210
eps: 0.17, clusters: 11, silhouette score: -0.171
eps: 0.18, clusters: 13, silhouette score: -0.194
eps: 0.19, clusters: 15, silhouette score: -0.215
eps: 0.20, clusters: 16, silhouette score: -0.222
eps:

The silouette score can helps to determine the optimal epsilon value.

In [14]:
min_samples_values = [2, 3, 4, 5, 6, 7, 8, 9, 10]

In [15]:
for min_samples in min_samples_values:
    # Apply DBSCAN
    dbscan = DBSCAN(eps=0.08, min_samples=min_samples, metric="cosine")
    clusters = dbscan.fit_predict(vectorized)

    # Analyze the clusters
    cluster_counts = Counter(clusters)
    num_clusters = len(cluster_counts) - (1 if -1 in cluster_counts else 0)
    num_noise = cluster_counts[-1] if -1 in cluster_counts else 0

    # Print the results
    print(f"Results for min_samples={min_samples}:")
    print(f"Number of clusters: {num_clusters}")
    print(f"Number of noise points: {num_noise}")
    print(f"Cluster distribution: {dict(cluster_counts)}\n")


Results for min_samples=2:
Number of clusters: 21
Number of noise points: 286
Cluster distribution: {0: 3061, 1: 3, -1: 286, 2: 2, 3: 3, 4: 2, 5: 5, 6: 2, 7: 4, 8: 3, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 2}

Results for min_samples=3:
Number of clusters: 6
Number of noise points: 316
Cluster distribution: {0: 3061, 1: 3, -1: 316, 2: 3, 3: 5, 4: 4, 5: 3}

Results for min_samples=4:
Number of clusters: 3
Number of noise points: 330
Cluster distribution: {0: 3056, -1: 330, 1: 5, 2: 4}

Results for min_samples=5:
Number of clusters: 3
Number of noise points: 343
Cluster distribution: {0: 3026, -1: 343, 1: 9, 2: 17}

Results for min_samples=6:
Number of clusters: 3
Number of noise points: 361
Cluster distribution: {0: 3008, -1: 361, 1: 9, 2: 17}

Results for min_samples=7:
Number of clusters: 3
Number of noise points: 367
Cluster distribution: {0: 3005, -1: 367, 1: 8, 2: 15}

Results for min_samples=8:
Number of clusters: 3
Number of noise points: 